### Importing neccessary packages

In [156]:
import tensorflow as tf
import keras
import numpy as np
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
from keras.models import Model, Sequential
from keras.layers import BatchNormalization
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten

### Getting data

In [157]:
path = 'dog vs cat\\train\\dog\\'
import cv2
X_train  =[]
images = []
for i in range(1,100):
    #exact = cv2.imread(path+str(i)+'.jpg',0)
    exact = path+str(i)+'.jpg'
    X_train.append(cv2.resize(cv2.imread(exact, 0), (28, 28)))
    #cv2.resize(exact,(28,28))
    images.append(exact)

### Converting to numpy array!

In [158]:
X_train = np.array(X_train)

In [159]:
X_train.shape

(99, 28, 28)

### Defining image parameters

In [160]:
#Define input image dimensions
#Large images take too much time and resources.
img_rows = 200
img_cols = 200
channels = 1
img_shape = (img_rows, img_cols, channels)

### Building Generator

In [161]:
def build_generator():

    noise_shape = (100,) #1D array of size 100 (latent vector / noise)

#Define your generator network 
#Here we are only using Dense layers. But network can be complicated based
#on the application. For example, you can use VGG for super res. GAN.         

    model = Sequential()

    model.add(Dense(256, input_shape=noise_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))

    model.summary()

    noise = Input(shape=noise_shape)
    img = model(noise)    #Generated image

    return Model(noise, img)

#Alpha — α is a hyperparameter which controls the underlying value to which the
#function saturates negatives network inputs.
#Momentum — Speed up the training





### Building Discriminator

In [162]:
##########################################################################

#Given an input image, the Discriminator outputs the likelihood of the image being real.
    #Binary classification - true or false (we're calling it validity)

def build_discriminator():


    model = Sequential()

    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)

    return Model(img, validity)
#The validity is the Discriminator’s guess of input being real or not.

### Pitting them against each other in model trainer

In [163]:
#Now that we have constructed our two models it’s time to pit them against each other.
#We do this by defining a training function, loading the data set, re-scaling our training
#images and setting the ground truths. 

def train(epochs, batch_size=128, save_interval=50):

    # Load the dataset
    
    #(X_train, _), (_, _) = mnist.load_data()
    path = 'dog vs cat\\train\\dog\\'
    import cv2
    X_train  =[]
    images = []
    
    for i in range(1,100):
        #exact = cv2.imread(path+str(i)+'.jpg',0)
        exact = path+str(i)+'.jpg'
        X_train.append(cv2.resize(cv2.imread(exact, 0), (200, 200)))
        images.append(exact)
        
    X_train = np.array(X_train)
    # Convert to float and Rescale -1 to 1 (Can also do 0 to 1)
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    
    
    #get images to same size and grayscale

#Add channels dimension. As the input to our gen and discr. has a shape 28x28x1.
    X_train = np.expand_dims(X_train, axis=3) 

    half_batch = int(batch_size / 2)


#We then loop through a number of epochs to train our Discriminator by first selecting
#a random batch of images from our true dataset, generating a set of images from our
#Generator, feeding both set of images into our Discriminator, and finally setting the
#loss parameters for both the real and fake images, as well as the combined loss. 
    
    for epoch in range(epochs):

        # ---------------------
        #  Train Discriminator
        # ---------------------

        # Select a random half batch of real images
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        imgs = X_train[idx]

 
        noise = np.random.normal(0, 1, (half_batch, 100))

        # Generate a half batch of fake images
        gen_imgs = generator.predict(noise)

        # Train the discriminator on real and fake images, separately
        #Research showed that separate training is more effective. 
        d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
        d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
    #take average loss from real and fake images. 
    #
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake) 

#And within the same loop we train our Generator, by setting the input noise and
#ultimately training the Generator to have the Discriminator label its samples as valid
#by specifying the gradient loss.
        # ---------------------
        #  Train Generator
        # ---------------------
#Create noise vectors as input for generator. 
#Create as many noise vectors as defined by the batch size. 
#Based on normal distribution. Output will be of size (batch size, 100)
        noise = np.random.normal(0, 1, (batch_size, 100)) 

        # The generator wants the discriminator to label the generated samples
        # as valid (ones)
        #This is where the genrator is trying to trick discriminator into believing
        #the generated image is true (hence value of 1 for y)
        valid_y = np.array([1] * batch_size) #Creates an array of all ones of size=batch size

        # Generator is part of combined where it got directly linked with the discriminator
        # Train the generator with noise as x and 1 as y. 
        # Again, 1 as the output as it is adversarial and if generator did a great
        #job of folling the discriminator then the output would be 1 (true)
        g_loss = combined.train_on_batch(noise, valid_y)


#Additionally, in order for us to keep track of our training process, we print the
#progress and save the sample image output depending on the epoch interval specified.  
# Plot the progress
        
        print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

        # If at save interval => save generated image samples
        if epoch % save_interval == 0:
            save_imgs(epoch)

#when the specific sample_interval is hit, we call the
#sample_image function. Which looks as follows.

### Saving images

In [164]:
def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, 100))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/dog_%d.png" % epoch)
    plt.close()
#This function saves our images for us to view


### Building and compiling the 2 models

In [165]:

##############################################################################

#Let us also define our optimizer for easy use later on.
#That way if you change your mind, you can change it easily here
optimizer = Adam(0.0002, 0.5)  #Learning rate and momentum.

# Build and compile the discriminator first. 
#Generator will be trained as part of the combined model, later. 
#pick the loss function and the type of metric to keep track.                 
#Binary cross entropy as we are doing prediction and it is a better
#loss function compared to MSE or other. 
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

#build and compile our Discriminator, pick the loss function

#SInce we are only generating (faking) images, let us not track any metrics.
generator = build_generator()
generator.compile(loss='binary_crossentropy', optimizer=optimizer)

##This builds the Generator and defines the input noise. 
#In a GAN the Generator network takes noise z as an input to produce its images.  
z = Input(shape=(100,))   #Our random input to the generator
img = generator(z)

#This ensures that when we combine our networks we only train the Generator.
#While generator training we do not want discriminator weights to be adjusted. 
#This Doesn't affect the above descriminator training.     
discriminator.trainable = False  

#This specifies that our Discriminator will take the images generated by our Generator
#and true dataset and set its output to a parameter called valid, which will indicate
#whether the input is real or not.  
valid = discriminator(img)  #Validity check on the generated image


#Here we combined the models and also set our loss function and optimizer. 
#Again, we are only training the generator here. 
#The ultimate goal here is for the Generator to fool the Discriminator.  
# The combined model  (stacked generator and discriminator) takes
# noise as input => generates images => determines validity

combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)


train(epochs=1000, batch_size=32, save_interval=50)

#Save model for future use to generate fake images
#Not tested yet... make sure right model is being saved..
#Compare with GAN4

generator.save('generator_model.h5')  #Test the model on GAN4_predict...
#Change epochs back to 30K
                
#Epochs dictate the number of backward and forward propagations, the batch_size
#indicates the number of training samples per backward/forward propagation, and the
#sample_interval specifies after how many epochs we call our sample_image function.

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_11 (Flatten)         (None, 40000)             0         
_________________________________________________________________
dense_71 (Dense)             (None, 512)               20480512  
_________________________________________________________________
leaky_re_lu_51 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_72 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_52 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_73 (Dense)             (None, 1)                 257       
Total params: 20,612,097
Trainable params: 20,612,097
Non-trainable params: 0
_________________________________________

95 [D loss: 1.170732, acc.: 65.62%] [G loss: 2.014693]
96 [D loss: 0.265237, acc.: 90.62%] [G loss: 2.777845]
97 [D loss: 0.388505, acc.: 81.25%] [G loss: 2.660751]
98 [D loss: 0.396670, acc.: 93.75%] [G loss: 1.966359]
99 [D loss: 0.841982, acc.: 71.88%] [G loss: 3.449025]
100 [D loss: 1.016982, acc.: 65.62%] [G loss: 1.939352]
101 [D loss: 1.226792, acc.: 65.62%] [G loss: 2.992561]
102 [D loss: 0.671807, acc.: 68.75%] [G loss: 4.034755]
103 [D loss: 0.607012, acc.: 78.12%] [G loss: 3.165904]
104 [D loss: 0.593604, acc.: 71.88%] [G loss: 2.393378]
105 [D loss: 0.361383, acc.: 84.38%] [G loss: 4.021498]
106 [D loss: 0.997189, acc.: 68.75%] [G loss: 3.029881]
107 [D loss: 0.587216, acc.: 71.88%] [G loss: 2.974076]
108 [D loss: 0.368687, acc.: 87.50%] [G loss: 3.385552]
109 [D loss: 1.486718, acc.: 65.62%] [G loss: 2.107825]
110 [D loss: 0.806941, acc.: 75.00%] [G loss: 2.241453]
111 [D loss: 0.317666, acc.: 84.38%] [G loss: 2.818464]
112 [D loss: 0.600983, acc.: 62.50%] [G loss: 1.94234

242 [D loss: 0.813587, acc.: 71.88%] [G loss: 5.842147]
243 [D loss: 1.781469, acc.: 62.50%] [G loss: 4.996121]
244 [D loss: 0.086832, acc.: 96.88%] [G loss: 5.840962]
245 [D loss: 0.699991, acc.: 75.00%] [G loss: 6.696988]
246 [D loss: 0.263682, acc.: 87.50%] [G loss: 7.083996]
247 [D loss: 0.747895, acc.: 59.38%] [G loss: 4.034898]
248 [D loss: 0.392411, acc.: 84.38%] [G loss: 4.008494]
249 [D loss: 0.198784, acc.: 90.62%] [G loss: 3.517772]
250 [D loss: 0.250086, acc.: 87.50%] [G loss: 4.382744]
251 [D loss: 0.599087, acc.: 81.25%] [G loss: 4.174688]
252 [D loss: 0.182925, acc.: 93.75%] [G loss: 4.531708]
253 [D loss: 0.632795, acc.: 87.50%] [G loss: 4.337148]
254 [D loss: 0.843401, acc.: 78.12%] [G loss: 4.969960]
255 [D loss: 2.039279, acc.: 56.25%] [G loss: 2.921168]
256 [D loss: 0.598585, acc.: 87.50%] [G loss: 5.220844]
257 [D loss: 0.206675, acc.: 93.75%] [G loss: 6.337654]
258 [D loss: 0.300929, acc.: 90.62%] [G loss: 3.226402]
259 [D loss: 0.254023, acc.: 87.50%] [G loss: 4.

389 [D loss: 0.216850, acc.: 93.75%] [G loss: 4.832414]
390 [D loss: 0.390899, acc.: 81.25%] [G loss: 5.790022]
391 [D loss: 0.336733, acc.: 84.38%] [G loss: 7.326101]
392 [D loss: 0.167693, acc.: 90.62%] [G loss: 7.147614]
393 [D loss: 0.175381, acc.: 90.62%] [G loss: 4.544852]
394 [D loss: 0.102740, acc.: 96.88%] [G loss: 5.195890]
395 [D loss: 0.167207, acc.: 93.75%] [G loss: 5.149321]
396 [D loss: 0.266002, acc.: 90.62%] [G loss: 8.310774]
397 [D loss: 0.167436, acc.: 90.62%] [G loss: 5.581613]
398 [D loss: 0.130576, acc.: 93.75%] [G loss: 5.263693]
399 [D loss: 0.198285, acc.: 90.62%] [G loss: 6.269851]
400 [D loss: 0.235138, acc.: 90.62%] [G loss: 5.691998]
401 [D loss: 0.077532, acc.: 93.75%] [G loss: 7.422054]
402 [D loss: 0.208409, acc.: 87.50%] [G loss: 5.666374]
403 [D loss: 0.140116, acc.: 96.88%] [G loss: 5.363342]
404 [D loss: 0.035752, acc.: 100.00%] [G loss: 4.572979]
405 [D loss: 0.117071, acc.: 96.88%] [G loss: 4.369845]
406 [D loss: 0.186148, acc.: 87.50%] [G loss: 6

535 [D loss: 0.024463, acc.: 100.00%] [G loss: 6.625327]
536 [D loss: 0.303764, acc.: 87.50%] [G loss: 4.675804]
537 [D loss: 0.090076, acc.: 96.88%] [G loss: 7.335235]
538 [D loss: 0.128857, acc.: 90.62%] [G loss: 5.653393]
539 [D loss: 0.062255, acc.: 100.00%] [G loss: 6.256038]
540 [D loss: 0.067299, acc.: 96.88%] [G loss: 6.218340]
541 [D loss: 0.068634, acc.: 96.88%] [G loss: 6.250668]
542 [D loss: 0.210394, acc.: 93.75%] [G loss: 5.853169]
543 [D loss: 0.175680, acc.: 96.88%] [G loss: 7.214812]
544 [D loss: 0.130521, acc.: 96.88%] [G loss: 5.115304]
545 [D loss: 0.164822, acc.: 93.75%] [G loss: 7.476113]
546 [D loss: 0.028954, acc.: 100.00%] [G loss: 7.456828]
547 [D loss: 0.381727, acc.: 84.38%] [G loss: 4.574229]
548 [D loss: 0.061491, acc.: 96.88%] [G loss: 5.657329]
549 [D loss: 0.023241, acc.: 100.00%] [G loss: 6.785613]
550 [D loss: 0.290298, acc.: 87.50%] [G loss: 6.581758]
551 [D loss: 0.062693, acc.: 100.00%] [G loss: 6.598353]
552 [D loss: 0.486040, acc.: 75.00%] [G los

681 [D loss: 0.177314, acc.: 93.75%] [G loss: 5.546850]
682 [D loss: 0.134855, acc.: 93.75%] [G loss: 4.877928]
683 [D loss: 0.067567, acc.: 100.00%] [G loss: 5.544353]
684 [D loss: 0.092975, acc.: 96.88%] [G loss: 6.612437]
685 [D loss: 0.228921, acc.: 87.50%] [G loss: 4.501318]
686 [D loss: 0.054889, acc.: 100.00%] [G loss: 6.144521]
687 [D loss: 0.096175, acc.: 96.88%] [G loss: 4.056989]
688 [D loss: 0.238210, acc.: 90.62%] [G loss: 4.656323]
689 [D loss: 0.162808, acc.: 90.62%] [G loss: 6.590250]
690 [D loss: 0.129810, acc.: 93.75%] [G loss: 8.334032]
691 [D loss: 0.135337, acc.: 93.75%] [G loss: 5.233853]
692 [D loss: 0.308306, acc.: 90.62%] [G loss: 4.836656]
693 [D loss: 0.166716, acc.: 93.75%] [G loss: 4.942737]
694 [D loss: 0.072371, acc.: 96.88%] [G loss: 6.389097]
695 [D loss: 0.058535, acc.: 100.00%] [G loss: 7.200727]
696 [D loss: 0.143039, acc.: 93.75%] [G loss: 4.361015]
697 [D loss: 0.148211, acc.: 90.62%] [G loss: 5.348376]
698 [D loss: 0.020612, acc.: 100.00%] [G loss

827 [D loss: 0.052779, acc.: 100.00%] [G loss: 6.027441]
828 [D loss: 0.021953, acc.: 100.00%] [G loss: 5.436787]
829 [D loss: 0.207061, acc.: 90.62%] [G loss: 4.639098]
830 [D loss: 0.053065, acc.: 96.88%] [G loss: 7.490411]
831 [D loss: 0.210538, acc.: 90.62%] [G loss: 4.793883]
832 [D loss: 0.140359, acc.: 93.75%] [G loss: 5.853099]
833 [D loss: 0.021065, acc.: 100.00%] [G loss: 6.819191]
834 [D loss: 0.226680, acc.: 90.62%] [G loss: 3.523521]
835 [D loss: 0.292067, acc.: 90.62%] [G loss: 5.300566]
836 [D loss: 0.037037, acc.: 100.00%] [G loss: 7.545459]
837 [D loss: 0.446435, acc.: 75.00%] [G loss: 6.520675]
838 [D loss: 0.020691, acc.: 100.00%] [G loss: 7.918078]
839 [D loss: 0.201781, acc.: 93.75%] [G loss: 6.373387]
840 [D loss: 0.077753, acc.: 93.75%] [G loss: 4.903471]
841 [D loss: 0.074709, acc.: 100.00%] [G loss: 4.710515]
842 [D loss: 0.099526, acc.: 96.88%] [G loss: 5.002622]
843 [D loss: 0.010038, acc.: 100.00%] [G loss: 4.869908]
844 [D loss: 0.160263, acc.: 93.75%] [G l

973 [D loss: 0.035877, acc.: 96.88%] [G loss: 5.276383]
974 [D loss: 0.130269, acc.: 96.88%] [G loss: 7.365512]
975 [D loss: 0.270882, acc.: 96.88%] [G loss: 5.847588]
976 [D loss: 0.051894, acc.: 96.88%] [G loss: 7.253478]
977 [D loss: 0.013924, acc.: 100.00%] [G loss: 7.302804]
978 [D loss: 0.001971, acc.: 100.00%] [G loss: 8.007173]
979 [D loss: 0.012679, acc.: 100.00%] [G loss: 7.230328]
980 [D loss: 0.221961, acc.: 93.75%] [G loss: 6.495571]
981 [D loss: 0.012799, acc.: 100.00%] [G loss: 6.792730]
982 [D loss: 0.105595, acc.: 93.75%] [G loss: 5.053909]
983 [D loss: 0.139243, acc.: 93.75%] [G loss: 4.874441]
984 [D loss: 0.043048, acc.: 100.00%] [G loss: 5.854867]
985 [D loss: 0.056930, acc.: 96.88%] [G loss: 6.445759]
986 [D loss: 0.005095, acc.: 100.00%] [G loss: 5.810003]
987 [D loss: 0.012759, acc.: 100.00%] [G loss: 6.367123]
988 [D loss: 0.092958, acc.: 96.88%] [G loss: 4.796904]
989 [D loss: 0.077075, acc.: 96.88%] [G loss: 5.492301]
990 [D loss: 0.008329, acc.: 100.00%] [G 